In [100]:
import os
import sys
import openai
import json
import requests
import time
from purreaddocs import *


In [101]:
from langchain.document_loaders import ReadTheDocsLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter


In [102]:
import bs4
import tiktoken
from uuid import uuid4
from tqdm.auto import tqdm
import openai

from langchain.document_loaders import ReadTheDocsLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

#loader = UnstructuredMarkdownLoader("C:/GitHub/test/")
loader = UnstructuredMarkdownLoader("C:/Users/roshnipatil/OneDrive - Microsoft/GitHub_new/test/abap-functions-deployment-guide.md")
docs = loader.load()
len(docs)

print("Loading data...")
pdf_folder_path = "C:/Users/roshnipatil/OneDrive - Microsoft/GitHub_new/test/"
print(os.listdir(pdf_folder_path))

my_file = "abap-functions-deployment-guide.md"
print(my_file)
my_string = my_file.replace(".md", "")
print(my_string)

# Load multiple files
# location of the pdf file/files. 
loaders = [UnstructuredMarkdownLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
print (len(loaders))
for loader in loaders:
    docs = loader.load()
    print (len(docs))
    print (docs[0])


Loading data...
['abap-functions-deployment-guide.md', 'asset-insights.md', 'catalog-asset-details.md', 'catalog-lineage-user-guide.md', 'catalog-permissions.md', 'catalog-private-link-account-portal.md', 'catalog-private-link-end-to-end.md', 'catalog-private-link-faqs.md', 'catalog-private-link-name-resolution.md', 'catalog-private-link-troubleshoot.md', 'catalog-private-link.md', 'classification-insights.md', 'concept-account-upgrade.md', 'concept-asset-normalization.md', 'concept-business-glossary.md', 'concept-elastic-data-map.md', 'concept-insights.md', 'concept-resource-sets.md', 'concept-scans-and-ingestion.md', 'concept-search.md', 'create-a-custom-classification-and-classification-rule.md', 'create-a-scan-rule-set.md', 'create-catalog-portal.md', 'create-catalog-powershell.md']
abap-functions-deployment-guide.md
abap-functions-deployment-guide
24
1
page_content='title: Metadata extraction ABAP function module in SAP R3 - Azure Purview\ndescription: This article outlines the st

In [103]:
#docs[0]
print(docs[0].page_content)
print(docs[0])
#print(docs[1])
#print(docs[2].page_content)

#docs[5].metadata['source'].replace('rtdocs/', 'https://')

title: 'Quickstart: Create an Azure Purview account using Azure PowerShell/Azure CLI (preview)'
description: This Quickstart describes how to create an Azure Purview account using Azure PowerShell/Azure CLI.
author: hophanms
ms.author: hophan
ms.date: 9/10/2021
ms.topic: quickstart
ms.service: purview
ms.custom:
  - mode-api

Customer intent: As a data steward, I want create a new Azure Purview Account so that I can scan and classify my data.

Quickstart: Create an Azure Purview account using Azure PowerShell/Azure CLI

In this Quickstart, you'll create an Azure Purview account using Azure PowerShell/Azure CLI.

[!IMPORTANT]
Azure Purview is currently in PREVIEW. The Supplemental Terms of Use for Microsoft Azure Previews include additional legal terms that apply to Azure features that are in beta, preview, or otherwise not yet released into general availability.

Azure Purview is a data governance service that helps you manage and govern your data landscape. By connecting to data acros

In [104]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [105]:
tiktoken.encoding_for_model('gpt-3.5-turbo')

token_counts = [tiktoken_len(doc.page_content) for doc in docs]

In [106]:
print(f"""Min: {min(token_counts)}
Avg: {int(sum(token_counts) / len(token_counts))}
Max: {max(token_counts)}""")

Min: 889
Avg: 889
Max: 889


In [107]:
from langchain.text_splitter import CharacterTextSplitter
#from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle
import os

print("Loading data...")
pdf_folder_path = "content/"
print(os.listdir(pdf_folder_path))

# Load multiple files
# location of the pdf file/files. 
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]


print(loaders)

alldocument = []
vectorstore = None
for loader in loaders:

    print("Loading raw document..." + loader.file_path)
    raw_documents = loader.load()

    print("Splitting text...")
    text_splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=800,
        chunk_overlap=100,
        length_function=len,
    )
    documents = text_splitter.split_documents(raw_documents)
    #alldocument = alldocument + documents

    print("Creating vectorstore...")
    embeddings = OpenAIEmbeddings()
    
    vectorstore = FAISS.from_documents(documents, embeddings)

    #with open("vectorstore.pkl", "wb") as f:
    with open("vectorstore.pkl", "ab") as f:
        pickle.dump(vectorstore, f)
        f.close()


Loading data...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'content/'

POST {Endpoint}/catalog/api/search/query?api-version=2023-02-01-preview

{
  "keywords": null,
  "limit": 10,
  "filter": {
  }
}
https://{accountName}.purview.azure.com


POST {Endpoint}/catalog/api/search/query?api-version=2023-02-01-preview

In [108]:
import requests

azuread_access_token=readtoken()

url = "https://roshPurview2.purview.azure.com/catalog/api/search/query?api-version=2023-02-01-preview"
data = { "keywords": None, "limit": 10, "filter": {}}
headers = {
    'Authorization': f'Bearer {azuread_access_token}',
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print("Request successful")
    print(response.json())
else:
    print("Request failed")
    print (response)


NameError: name 'readtoken' is not defined